In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[10]:

import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
import os

from os.path import join as ospj
from os.path import expanduser
from munch import Munch as mch
import numpy as np
from tqdm import tqdm

import clip

from ds import prepare_coco_dataloaders, prepare_flickr_dataloaders, prepare_cub_dataloaders, prepare_flo_dataloaders
import torch.distributions as dist

import matplotlib.pyplot as plt

In [91]:
from pycocotools.coco import COCO
import numpy as np
import os
import json


def _get_coco_file_paths(dataset_root):
    """Select proper train / val classes and omit id files."""
    
    # Load the existing ID arrays
    train_ids = np.load('../ProbVLM/src/ds/annotations/coco/coco_train_ids.npy')
    train_extra_ids = np.load('../ProbVLM/src/ds/annotations/coco/coco_restval_ids.npy')
    val_ids = np.load('../ProbVLM/src/ds/annotations/coco/coco_dev_ids.npy')[:5000]
    te_ids = np.load('../ProbVLM/src/ds/annotations/coco/coco_test_ids.npy')

    

    # Image root and annotation file paths
    image_root = os.path.join(dataset_root, 'images/tmp')  # train + valid
    train_ann_path = os.path.join(dataset_root, 'captions_train2014.json')
    val_ann_path = os.path.join(dataset_root, 'captions_val2014.json')

    
    
    return (train_ids, train_extra_ids, val_ids, te_ids,
            image_root, train_ann_path, val_ann_path)


In [92]:
dataset_root = '../ProbVLM/Datasets/coco'  # Replace with your actual dataset root path
train_ids, train_extra_ids, val_ids, te_ids, image_root, train_ann_path, val_ann_path = _get_coco_file_paths(dataset_root)

In [213]:
from pycocotools.coco import COCO

# Initialize COCO API for caption annotations
# Change to either val or train to add the CLIP prompts like
annFile = '../ProbVLM/Datasets/coco/captions_val2014.json'  # Path to COCO annotations file
coco = COCO(annFile)

# Get all available annotation IDs
all_annotation_ids = set(coco.getAnnIds())
sorted_ann_id = sorted(all_annotation_ids)

# check the last caption id
valid_annotation_id = sorted_ann_id[-1]

# Load and print the annotation for the selected ID
annotation = coco.loadAnns(valid_annotation_id)
print("Annotation for ID", valid_annotation_id, ":", annotation)

loading annotations into memory...
Done (t=0.28s)
creating index...
index created!
Annotation for ID 829718 : [{'image_id': 388215, 'id': 829718, 'caption': 'A young man is in blue and white playing tennis.'}]


In [217]:
len(human_image_ids)+829718

851352

In [214]:
# function adds clips prompt like to the annotation file
def filter_annotations(captions, image_ids):
    text_list =  ["an image of a person", "a photo of a person"] 
    for i in range(len(text_list)):
        for j, img in enumerate(image_ids):
            ann =  {
                'image_id': img,
                'id': 829719  + j,  # Ensure unique ID
                'caption': text_list[i]
            }
            captions['annotations'].append(ann)


In [218]:
# change to train or val
train_ann_path = '../ProbVLM/Datasets/coco/captions_val2014.json'
with open(train_ann_path) as train_file:
        train_captions = json.load(train_file)
        
filter_annotations(train_captions, human_image_ids)

In [203]:
def convert_ids(obj):
        if isinstance(obj, dict):
            return {key: (int(value) if isinstance(value, (np.int64, np.float64)) else value) for key, value in obj.items()}
        return obj

In [219]:
# save annotation file
train_captions['annotations'] = [convert_ids(ann) for ann in train_captions['annotations']]
extended_train_ann_path = '../ProbVLM/Datasets/coco/captions_val2014_extra.json'
with open(extended_train_ann_path, 'w') as train_file:
        json.dump(train_captions, train_file, indent=4)

In [220]:
# Open and load the JSON data
extended_train_ann_path = '../ProbVLM/Datasets/coco/captions_val2014_extra.json'
with open(extended_train_ann_path, 'r') as file:
    data = json.load(file)
    
data

{'info': {'description': 'COCO 2014 Dataset',
  'url': 'http://cocodataset.org',
  'version': '1.0',
  'year': 2014,
  'contributor': 'COCO Consortium',
  'date_created': '2017/09/01'},
 'images': [{'license': 3,
   'file_name': 'COCO_val2014_000000391895.jpg',
   'coco_url': 'http://images.cocodataset.org/val2014/COCO_val2014_000000391895.jpg',
   'height': 360,
   'width': 640,
   'date_captured': '2013-11-14 11:18:45',
   'flickr_url': 'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg',
   'id': 391895},
  {'license': 4,
   'file_name': 'COCO_val2014_000000522418.jpg',
   'coco_url': 'http://images.cocodataset.org/val2014/COCO_val2014_000000522418.jpg',
   'height': 480,
   'width': 640,
   'date_captured': '2013-11-14 11:38:44',
   'flickr_url': 'http://farm1.staticflickr.com/1/127244861_ab0c0381e7_z.jpg',
   'id': 522418},
  {'license': 3,
   'file_name': 'COCO_val2014_000000184613.jpg',
   'coco_url': 'http://images.cocodataset.org/val2014/COCO_val2014_000000184613.

In [60]:
# open either val or train ids
val_ids = np.load('../ProbVLM/src/ds/annotations/coco/coco_dev_ids.npy')[:5000]

In [61]:
# get extended ann file
import json
extended_train_ann_path = '../ProbVLM/Datasets/coco/captions_val2014_extra.json'
with open(extended_train_ann_path, 'r') as file:
    data = json.load(file)
    
data

{'info': {'description': 'COCO 2014 Dataset',
  'url': 'http://cocodataset.org',
  'version': '1.0',
  'year': 2014,
  'contributor': 'COCO Consortium',
  'date_created': '2017/09/01'},
 'images': [{'license': 3,
   'file_name': 'COCO_val2014_000000391895.jpg',
   'coco_url': 'http://images.cocodataset.org/val2014/COCO_val2014_000000391895.jpg',
   'height': 360,
   'width': 640,
   'date_captured': '2013-11-14 11:18:45',
   'flickr_url': 'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg',
   'id': 391895},
  {'license': 4,
   'file_name': 'COCO_val2014_000000522418.jpg',
   'coco_url': 'http://images.cocodataset.org/val2014/COCO_val2014_000000522418.jpg',
   'height': 480,
   'width': 640,
   'date_captured': '2013-11-14 11:38:44',
   'flickr_url': 'http://farm1.staticflickr.com/1/127244861_ab0c0381e7_z.jpg',
   'id': 522418},
  {'license': 3,
   'file_name': 'COCO_val2014_000000184613.jpg',
   'coco_url': 'http://images.cocodataset.org/val2014/COCO_val2014_000000184613.

In [62]:
# get all caption ids including the added ones
caption_ids = [img['id'] for img in data['annotations']]
caption_ids[-1]

851352

In [63]:
# open og annotation file
extended_train_ann_path1 = '../ProbVLM/Datasets/coco/captions_val2014.json'
with open(extended_train_ann_path1, 'r') as file:
    data1 = json.load(file)
    
data1

{'info': {'description': 'COCO 2014 Dataset',
  'url': 'http://cocodataset.org',
  'version': '1.0',
  'year': 2014,
  'contributor': 'COCO Consortium',
  'date_created': '2017/09/01'},
 'images': [{'license': 3,
   'file_name': 'COCO_val2014_000000391895.jpg',
   'coco_url': 'http://images.cocodataset.org/val2014/COCO_val2014_000000391895.jpg',
   'height': 360,
   'width': 640,
   'date_captured': '2013-11-14 11:18:45',
   'flickr_url': 'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg',
   'id': 391895},
  {'license': 4,
   'file_name': 'COCO_val2014_000000522418.jpg',
   'coco_url': 'http://images.cocodataset.org/val2014/COCO_val2014_000000522418.jpg',
   'height': 480,
   'width': 640,
   'date_captured': '2013-11-14 11:38:44',
   'flickr_url': 'http://farm1.staticflickr.com/1/127244861_ab0c0381e7_z.jpg',
   'id': 522418},
  {'license': 3,
   'file_name': 'COCO_val2014_000000184613.jpg',
   'coco_url': 'http://images.cocodataset.org/val2014/COCO_val2014_000000184613.

In [64]:
# get og caption files (without the extra prompts)
caption_ids1 = [img['id'] for img in data1['annotations']]
caption_ids1[-1]

829317

In [65]:
851352-829317

22035

In [66]:
caption_ids_new = [ident for ident in caption_ids if ident> 829317]

In [67]:
# obtain the caption ids of the extra annotations
caption_ids = set(caption_ids)  # Convert to set for fast lookups
caption_ids1 = set(caption_ids1)

# Find IDs in annotation_ids but not in val_ids
diff_ids = caption_ids - caption_ids1  # Set difference

# Convert back to list if needed
diff_ids = list(diff_ids)

In [68]:
diff_ids[-1]

851352

In [70]:
list(val_ids)[-1]

214156

Extend the caption ids to include the added annotations:

In [71]:
train_ids_list = list(val_ids)

In [72]:
train_ids_list.extend(diff_ids)

In [73]:
train_ids_list = np.array(train_ids_list)
train_ids_list

array([474921, 479322, 479334, ..., 851350, 851351, 851352])

In [74]:
train_ids_list[-1]

851352

In [75]:
np.save('../ProbVLM/src/ds/annotations/coco/coco_dev_ids_extra.npy', train_ids_list)